In [1]:
import os
import time
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import ShuffleSplit, KFold

# train = pd.read_csv("C:/Users/swl3guest/CE4032/datasets/train_modified_v2.csv")
# test = pd.read_csv("C:/Users/swl3guest/CE4032/datasets/test_modified_v2.csv")


In [2]:
train_df = pd.read_csv("../datasets/modified_train.csv")
test_df = pd.read_csv("../datasets/modified_test.csv")
ids = test_df['TRIP_ID']
train_df.head()

,TRIP_ID,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DATE,END_TIME,DAY_OF_WEEK,HOUR,ORIGIN_LNG,...,CUM_DIST,MEDIAN_VELOCITY,FINAL_VELOCITY,CALL_TYPE_A,CALL_TYPE_B,CALL_TYPE_C,ACTUAL_DAYTYPE_A,ACTUAL_DAYTYPE_B,ACTUAL_DAYTYPE_C,DURATION
0,T1,NaN,15.0,20000542,1408039037,2014-08-14 18:57:17+01:00,1408039187,4,1,-8.585676,...,359.965114,2.314499,3.277962,0,1,0,0,0,1,150
1,T1,NaN,15.0,20000542,1408039037,2014-08-14 18:57:17+01:00,1408039187,4,1,-8.585676,...,239.679711,1.610086,6.630079,0,1,0,0,0,1,150
2,T1,NaN,15.0,20000542,1408039037,2014-08-14 18:57:17+01:00,1408039187,4,1,-8.585676,...,140.228519,1.521722,2.508781,0,1,0,0,0,1,150
3,T2,NaN,57.0,20000108,1408038611,2014-08-14 18:50:11+01:00,1408039196,4,1,-8.610876,...,1931.909561,4.053948,0.389853,0,1,0,0,0,1,585
4,T2,NaN,57.0,20000108,1408038611,2014-08-14 18:50:11+01:00,1408039196,4,1,-8.610876,...,1943.509739,3.147156,0.348773,0,1,0,0,0,1,585


In [3]:
train_df['DURATION'] = train_df['DURATION'] / 15
y= np.log(train_df['DURATION']) + 1
train_cl = train_df.drop(columns=['TRIP_ID','TIMESTAMP','DATE', 'END_TIME',  'DURATION', 'ORIGIN_CALL',	'ORIGIN_STAND'])
test_cl= test_df.drop(columns=[ 'TRIP_ID','TIMESTAMP','DATE', 'END_TIME', 'ORIGIN_CALL',	'ORIGIN_STAND'])
train_cl.head()

,TAXI_ID,DAY_OF_WEEK,HOUR,ORIGIN_LNG,ORIGIN_LAT,ORIGIN_HEADER,ORIGIN_DISTANCE_TO_CC,CUT_OFF_LENGTH,CUT_OFF_LAT,CUT_OFF_LNG,...,HEADER_ORIGIN_TO_CUT_OFF,CUM_DIST,MEDIAN_VELOCITY,FINAL_VELOCITY,CALL_TYPE_A,CALL_TYPE_B,CALL_TYPE_C,ACTUAL_DAYTYPE_A,ACTUAL_DAYTYPE_B,ACTUAL_DAYTYPE_C
0,20000542,4,1,-8.585676,41.148522,-1.182260,2691.258243,10,41.146623,-8.584884,...,2.837288,359.965114,2.314499,3.277962,0,1,0,0,0,1
1,20000542,4,1,-8.585676,41.148522,-1.182260,2691.258243,8,41.147460,-8.586999,...,-2.388128,239.679711,1.610086,6.630079,0,1,0,0,0,1
2,20000542,4,1,-8.585676,41.148522,-1.182260,2691.258243,7,41.147847,-8.586324,...,-2.515665,140.228519,1.521722,2.508781,0,1,0,0,0,1
3,20000108,4,1,-8.610876,41.145570,-0.273847,1399.904997,32,41.161266,-8.604090,...,0.314672,1931.909561,4.053948,0.389853,0,1,0,0,0,1
4,20000108,4,1,-8.610876,41.145570,-0.273847,1399.904997,35,41.161275,-8.604135,...,0.312552,1943.509739,3.147156,0.348773,0,1,0,0,0,1


In [4]:
test_cl.head()

,TAXI_ID,DAY_OF_WEEK,HOUR,ORIGIN_LNG,ORIGIN_LAT,ORIGIN_HEADER,ORIGIN_DISTANCE_TO_CC,CUT_OFF_LENGTH,CUT_OFF_LAT,CUT_OFF_LNG,...,HEADER_ORIGIN_TO_CUT_OFF,CUM_DIST,MEDIAN_VELOCITY,FINAL_VELOCITY,CALL_TYPE_A,CALL_TYPE_B,CALL_TYPE_C,ACTUAL_DAYTYPE_A,ACTUAL_DAYTYPE_B,ACTUAL_DAYTYPE_C
0,20000542,3,18,-8.585676,41.148522,-1.182260,2691.258243,11,41.146623,-8.584884,...,2.837288,513.623887,2.411640,10.243918,0,1,0,0,0,1
1,20000108,3,18,-8.610876,41.145570,-0.273847,1399.904997,40,41.163597,-8.601894,...,0.358853,2282.133126,3.340840,5.146115,0,1,0,0,0,1
2,20000370,3,18,-8.585739,41.148558,-1.182897,2684.853296,40,41.167719,-8.574903,...,0.402469,3911.607406,5.660043,6.645085,0,1,0,0,0,1
3,20000492,3,18,-8.613963,41.141169,-0.065131,1841.553918,8,41.140980,-8.614638,...,-1.926755,176.758074,1.064723,2.607521,0,1,0,0,0,1
4,20000621,3,18,-8.619903,41.148036,0.338632,1137.845589,2,41.148036,-8.619894,...,1.570796,0.754424,0.050295,0.050295,0,1,0,0,0,1


In [5]:
train = train_cl

In [6]:
test = test_cl
feature_names = list(train.columns)

In [7]:
X = np.array(train, dtype=np.float)
X_test = np.array(test, dtype=np.float)

In [8]:
clf = GradientBoostingRegressor(n_estimators=10, learning_rate = 0.15, max_depth=3, max_features = 15, min_weight_fraction_leaf = 0.15, random_state=21, verbose = 1)
clf.fit(X, y)

      Iter       Train Loss   Remaining Time 
         1           0.7389            3.03s
         2           0.6398            1.36s
         3           0.5670            0.81s
         4           0.5235            0.53s
         5           0.4857            0.35s
         6           0.4519            0.24s
         7           0.4274            0.15s
         8           0.4115            0.09s
         9           0.3998            0.04s
        10           0.3912            0.00s


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.15, loss='ls', max_depth=3,
                          max_features=15, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.15, n_estimators=10,
                          n_iter_no_change=None, presort='auto',
                          random_state=21, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=1, warm_start=False)

In [9]:
logytest = clf.predict(X_test)
ytest = (np.exp(logytest) -1 )
print('Test shape OK.') if test.shape[0] == ytest.shape[0] else print('Oops')
result= pd.DataFrame({"TRIP_ID": test_df["TRIP_ID"], "TRAVEL_TIME": ytest*15})
result.to_csv("../datasets/gradient_boost.csv", index=False)

Test shape OK.


In [10]:
print('training a random forest regressor ...')
# Initialize the famous Random Forest Regressor from scikit-learn
clf = RandomForestRegressor(n_estimators=10, max_depth = 3, min_weight_fraction_leaf= 0.15, max_features = 15, n_jobs=-1, random_state=21,verbose = 1)
clf.fit(X, y)

training a random forest regressor ...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
                      max_features=15, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.15, n_estimators=10, n_jobs=-1,
                      oob_score=False, random_state=21, verbose=1,
                      warm_start=False)

In [11]:
logytest = clf.predict(X_test)
ytest = (np.exp(logytest) -1 )
print('Test shape OK.') if test.shape[0] == ytest.shape[0] else print('Oops')
result= pd.DataFrame({"TRIP_ID": test_df["TRIP_ID"], "TRAVEL_TIME": ytest*15})
result.to_csv("../datasets/random_forest.csv", index=False)

Test shape OK.


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
